<a href="https://colab.research.google.com/github/OlegV12/Neural/blob/Lesson_4/Lesson_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Практическое задание

Вариант 1. (простой)

- обучить сверточную нейронную сеть в стиле AlexNet (с падением размера ядра свертки и последовательностью блоков свертка-пулинг  (conv-pool)-(conv-pool)-...) на датасете fashion-mnist или cifar10
- оценить рост точности при увеличении ширины сети (больше ядер)
- оценить рост точности при увеличении глубины сети (больше слоев)
- (опциционально)сравнить с точностью полносвязной сети для этой выборки
    </li>

Вариант 2. (сложный)
- реализовать нейронную сеть или стек из сверток (Concatenate) на сifar10.
- оценить рост точности при увеличении ширины сети (больше ядер), больше нитей.
- оценить рост точности при увеличении глубины сети (больше слоев)
    </li>
</ol>

In [1]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical

In [2]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# нормализация данных [0, 1]
x_train /= 255 
x_test /= 255 

# трансформация лейблов в one-hot encoding
y_train = to_categorical(y_train, 10) 
y_test = to_categorical(y_test, 10) 

# изменение размерности массива в 4D массив
x_train = x_train.reshape(x_train.shape[0], 28,28,1)
x_test = x_test.reshape(x_test.shape[0], 28,28,1)

In [3]:
x_train.shape

(60000, 28, 28, 1)

In [4]:
model = Sequential()
model.add(layers.Input(shape=(28,28,1)))
model.add(layers.Conv2D(6, kernel_size=(5, 5),  padding='same', activation='tanh',))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
model.add(layers.Conv2D(16, kernel_size=(5, 5), padding='same', activation='tanh',))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.Dense(10, activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 6)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 16)        2416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 16)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 2704)              0         
                                                                 
 dense (Dense)               (None, 84)                2

In [6]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])

In [7]:
hist = model.fit(x=x_train,y=y_train, epochs=2, batch_size=128, validation_data=(x_test, y_test), verbose=1)
test_score = model.evaluate(x_test, y_test)

Epoch 1/2
469/469 [==============================] - 8s 12ms/step - loss: 0.8354 - accuracy: 0.7237 - val_loss: 0.5941 - val_accuracy: 0.7774
Epoch 2/2
313/313 [==============================] - 1s 4ms/step - loss: 0.5142 - accuracy: 0.8108


In [8]:
# Увеличим количество ядер
model_1 = Sequential()
model_1.add(layers.Input(shape=(28,28,1)))
model_1.add(layers.Conv2D(12, kernel_size=(5, 5),  padding='same', activation='tanh',))
model_1.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
model_1.add(layers.Conv2D(32, kernel_size=(5, 5), padding='same', activation='tanh',))
model_1.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model_1.add(layers.Flatten())
model_1.add(layers.Dense(168, activation='tanh'))
model_1.add(layers.Dense(10, activation='softmax'))


In [9]:
model_1.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 12)        312       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 27, 27, 12)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 27, 27, 32)        9632      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 5408)              0         
                                                                 
 dense_2 (Dense)             (None, 168)              

In [10]:
model_1.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])
hist = model_1.fit(x=x_train,y=y_train, epochs=2, batch_size=128, validation_data=(x_test, y_test), verbose=1)
test_score = model_1.evaluate(x_test, y_test)

Epoch 1/2
469/469 [==============================] - 7s 13ms/step - loss: 0.7866 - accuracy: 0.7369 - val_loss: 0.5629 - val_accuracy: 0.7891
Epoch 2/2
313/313 [==============================] - 1s 4ms/step - loss: 0.4837 - accuracy: 0.8241


In [11]:
# добавим 2 слоя (conv+pool)
model_2 = Sequential()
model_2.add(layers.Input(shape=(28,28,1)))
model_2.add(layers.Conv2D(6, kernel_size=(3, 3),  padding='same', activation='tanh',))
model_2.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
model_2.add(layers.Conv2D(16, kernel_size=(3, 3), padding='same', activation='tanh',))
model_2.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model_2.add(layers.Conv2D(16, kernel_size=(3, 3), padding='same', activation='tanh',))
model_2.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model_2.add(layers.Conv2D(16, kernel_size=(3, 3), padding='same', activation='tanh',))
model_2.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))


model_2.add(layers.Flatten())
model_2.add(layers.Dense(3000, activation='tanh'))
model_2.add(layers.Dense(150, activation='tanh'))
model_2.add(layers.Dense(10, activation='softmax'))
model_2.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])

In [12]:
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 28, 28, 6)         60        
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 27, 27, 6)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 27, 27, 16)        880       
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 13, 13, 16)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 13, 13, 16)        2320      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 6, 6, 16)        

In [13]:
hist = model_2.fit(x=x_train,y=y_train, epochs=2, batch_size=128, validation_data=(x_test, y_test), verbose=1)
test_score = model_2.evaluate(x_test, y_test)

Epoch 1/2
469/469 [==============================] - 6s 12ms/step - loss: 1.2763 - accuracy: 0.5991 - val_loss: 0.7892 - val_accuracy: 0.7175
Epoch 2/2
313/313 [==============================] - 1s 4ms/step - loss: 0.6572 - accuracy: 0.7579


Accuracy снизилась

In [14]:
final_model = Sequential([
                        layers.Input(shape=(28,28,1)),
                        layers.Flatten(),
                        layers.Dense(50, activation='relu',),
                        layers.Dense(50, activation='tanh'),
                        layers.Dense(50, activation='tanh'),
                        layers.Dense(50, activation='tanh'),

                        layers.Dense(10, activation='softmax'),
                        ])


final_model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    )
hh = final_model.fit(x=x_train,y=y_train, epochs=2, batch_size=128, validation_data=(x_test, y_test), verbose=1)
test_score = model_2.evaluate(x_test, y_test)

Epoch 1/2
469/469 [==============================] - 3s 5ms/step - loss: 0.5661 - accuracy: 0.7991 - val_loss: 0.4571 - val_accuracy: 0.8357
Epoch 2/2
313/313 [==============================] - 1s 4ms/step - loss: 0.6572 - accuracy: 0.7579


In [15]:
final_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_7 (Dense)             (None, 50)                39250     
                                                                 
 dense_8 (Dense)             (None, 50)                2550      
                                                                 
 dense_9 (Dense)             (None, 50)                2550      
                                                                 
 dense_10 (Dense)            (None, 50)                2550      
                                                                 
 dense_11 (Dense)            (None, 10)                510       
                                                                 
Total params: 47,410
Trainable params: 47,410
Non-trai

val_accuracy на полносвязной модели получили немного лучше